In [1]:
import os
import random
random.seed(99)
import shutil

import numpy as np
np.random.seed(99)

In [2]:
IS_RESET = True
SRC_DATA = './dataset/img_data_152'
DST_DATA = './federated_data'
CLIENTS = 15 # 1 is validation set (index 0)
VAL_RATE = 0.1
# balanced_iid balanced_noniid unbalanced_iid unbalanced_noniid as_is
TYPE = 'iid'

In [3]:
def get_files(root_dir, ext=('.jpg', '.png')):
    queue = [root_dir]
    while len(queue) != 0:
        nest_dir = queue.pop()
        label = None
        files = list()
        with os.scandir(nest_dir) as it:
            for entry in it:
                if not entry.name.startswith('.') and entry.is_file():
                    if entry.name.endswith(ext):
                        label = os.path.basename(os.path.dirname(entry.path)) # dirname is label
                        files.append(entry.path)
                elif not entry.name.startswith('.') and entry.is_dir():
                    queue.append(entry.path)
            if label is not None:
                yield label, files

In [4]:
data_root = os.path.abspath(os.path.expanduser(SRC_DATA))
dst_base = os.path.abspath(os.path.expanduser(DST_DATA))

if IS_RESET:
    if os.path.exists(dst_base):
        shutil.rmtree(dst_base)

for label, files in get_files(data_root):
    random.shuffle(files)
    nfile = len(files)
    # copy validation set
    nval = int(nfile*VAL_RATE)
    dst = os.path.join(dst_base, '0', label)
    os.makedirs(dst, exist_ok=True)
    for path in files[:nval]:
        shutil.copy(path, dst)
    files = files[nval:]
    # copy trainset
    ntrain = (nfile - nval)//CLIENTS
    for i in range(1, CLIENTS+1):
        dst = os.path.join(dst_base, str(i), label)
        os.makedirs(dst, exist_ok=True)
        for path in files[:ntrain]:
            shutil.copy(path, dst)
        files = files[ntrain:]
    print(f'Done {label}')

Done vimeo
Done icqchat
Done voipbuster
Done scp
Done ftps
Done sftp
Done youtube
Done aim
Done hangout
Done netflix
Done torrent
Done spotify
Done skype
Done email
Done facebook
Done gmail
